In [ ]:
# !cp -r /kaggle/input/nsfw-dataset1/nsfw_dataset_v1 /kaggle/working/

In [ ]:
# !cp -r /kaggle/input/nude-dataset/final_dataset_nood /kaggle/working/

In [ ]:
!cp -r /kaggle/input/final-nude-dataset/final_dataset_nood /kaggle/working/

In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import numpy as np
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!git clone https://github.com/emiliantolo/pytorch_nsfw_model.git

In [ ]:
%cd pytorch_nsfw_model

In [ ]:
%ls

In [ ]:
# %cd ../

In [ ]:
transformations = transforms.Compose([
transforms.Resize([224,224]),
transforms.CenterCrop(224),
transforms.ToTensor(),
# transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# train_set = datasets.ImageFolder("../nsfw_dataset_v1/train", transform = transformations)
# test_set = datasets.ImageFolder("../nsfw_dataset_v1/test", transform = transformations)

# train_set = datasets.ImageFolder("final_dataset_nood/train", transform = transformations)
# test_set = datasets.ImageFolder("final_dataset_nood/test", transform = transformations)

train_set = datasets.ImageFolder("../final_dataset_nood/train", transform = transformations)
test_set = datasets.ImageFolder("../final_dataset_nood/test", transform = transformations)

In [ ]:
# drawings, hentai, neutral, porn, sexy = [0,0,0,0,0]
# for ind, el in train_set:
#     if el==0:drawings+=1
#     elif el==1:hentai+=1
#     elif el==2:neutral+=1
#     elif el==3:porn+=1
#     elif el==4:sexy+=1
# print(drawings, hentai, neutral, porn, sexy)

In [ ]:
sizes_datasets = {'train_data': len(train_set), 'validation_data': len(test_set)}

In [ ]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=8, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size =8, shuffle=True)

In [ ]:
# print(train_loader.class_to_inx)

In [ ]:
loaders_data = {'train_data': train_loader, 'validation_data': test_loader}

In [ ]:
obr = transforms.Compose([transforms.ToPILImage()])
img_collect=[]
count = 0
for i,l in train_loader:
    collect = []
    print(l)
    for img in i:
        collect.append(obr(img))
    img_collect.append(collect)
    if count > 2:
        break
    count+=1

In [ ]:
img_collect[0][0]

In [ ]:
# #model classes
# # classes=['drawings', 'hentai', 'neutral', 'porn', 'sexy']
# classes=['nude', 'safe_people', 'safe']
# import torchvision.transforms as T
# #load images
# entries = test_loader

# for content,batch in entries:
    
#     fig=plt.figure(figsize=(10,10))
#     i=0
    
#     for entry in content:

#         #prediction
#         index = batch[i]
        
#         image = T.ToPILImage()(entry)
#         i+=1
#         sub = fig.add_subplot(1, len(batch), i)
#         sub.set_title(classes[index])
#         plt.axis('off')
#         plt.imshow(image)
#     plt.show()

In [ ]:
img_collect[0][2]

In [ ]:
from __future__ import print_function, division

import torch

import torch.nn as nn

import torch.optim as optim

from torch.optim import lr_scheduler, Adam

import numpy as np

import torchvision
from torchvision import datasets, models, transforms

import matplotlib.pyplot as plt

import time

import os

import copy

plt.ion()   # This is the interactive mode

In [ ]:
class_names = train_set.classes

In [ ]:
class_names

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
train_loss = []
test_loss = []
train_acc = []
test_acc = []
# scheduler = lr_scheduler

def model_training(res_model, criterion, optimizer, number_epochs=25):
    since = time.time()
    best_resmodel_wts = copy.deepcopy(res_model.state_dict())
    best_accuracy = 0.0
    for epochs in range(number_epochs):
        print('Epoch {}/{}'.format(epochs, number_epochs - 1))
        print('-' * 10)
        for phase in ['train_data', 'validation_data']: ## Here each epoch is having a training and validation phase
            if phase == 'train_data':
                res_model.train()  ## Here we are setting our model to training mode
            else:
                res_model.eval()   ## Here we are setting our model to evaluate mode
            running_loss = 0.0
            running_corrects = 0
            answ = np.array([])
            pred = np.array([])
            for inputs, labels in loaders_data[phase]: ## Iterating over data.
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad() ## here we are making the gradients to zero
                with torch.set_grad_enabled(phase == 'train_data'): ## forwarding and then tracking the history if only in train
                    outputs = res_model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if phase == 'train_data': 
                        loss.backward()
                        optimizer.step()
                    answ = np.append(answ,np.array(labels.cpu()))
                    pred = np.append(pred,np.array(preds.cpu()))
               #

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

#             if phase == 'train_data':
#                 scheduler.step()
#['drawings', 'hentai', 'neutral', 'porn', 'sexy']

            ConfusionMatrixDisplay(confusion_matrix(answ,pred),display_labels=
                                    ['nude', 'safe_people', 'safe']).plot()
#                           ['drawings', 'hentai', 'neutral', 'porn', 'sexy']).plot()
                                   #['nude', 'safe_people', 'safe']).plot()
            plt.show()
            epoch_loss = running_loss / sizes_datasets[phase]
            epoch_acc = running_corrects.double() / sizes_datasets[phase]
            if phase == 'train_data':
                train_loss.append(epoch_loss)
                train_acc.append(epoch_acc)
            else:
                test_loss.append(epoch_loss)
                test_acc.append(epoch_acc) 
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))



            if phase == 'validation_data' and epoch_acc > best_accuracy: ## deep copy the model
                
                print('Cкопировано!')

                best_accuracy = epoch_acc

                best_resmodel_wts = copy.deepcopy(res_model.state_dict())
    print()
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_accuracy))
     # load best model weights

    res_model.load_state_dict(best_resmodel_wts)

    return res_model

In [ ]:
torch.cuda.empty_cache()

In [ ]:
finetune_model = models.resnet50(pretrained=True)

num_ftrs = finetune_model.fc.in_features


finetune_model.fc = nn.Linear(num_ftrs, 3)
# nn.Sequential(nn.Linear(num_ftrs, 5),
#           nn.Softmax())


finetune_model = finetune_model.to(device)


criterion = nn.NLLLoss()


finetune_optim = optim.Adam(finetune_model.parameters(),lr=10e-4)

In [ ]:
# finetune_model = model_training(finetune_model, criterion, finetune_optim,
#                        number_epochs=25)

In [ ]:
# %pip install -U git+https://github.com/platelminto/NudeNet

In [ ]:
# from nudenet import NudeClassifier

# model = NudeClassifier()

In [ ]:
# model.classify('/kaggle/input/probe-set/probe/00b45f1073a9ed79356b655ac83275b971933109a8fda152c0bcdacd9ca89218.jpg')

In [ ]:
# model.classify('/kaggle/input/probe-set/probe/0a01c8d3b32715885ab43cd738ee74c96ebe78ca9bc4b69a3be061101329f8c5.jpg')

In [ ]:
# model.classify('/kaggle/input/probe-set/probe/0a0f062cf6c12acd0cb85d477591665f8a33a21398ede93ea7addce3cda6b971.jpg')

In [ ]:
# model.classify('/kaggle/input/probe-set/probe/0acba7b1dd82ea51bf9820b535704d821d003259189cd40aa15ea23a54bfa4fc.jpg')

In [ ]:
# model.classify('/kaggle/input/probe-set/probe/0acba7b1dd82ea51bf9820b535704d821d003259189cd40aa15ea23a54bfa4fc.jpg')

In [ ]:
# !git clone https://github.com/emiliantolo/pytorch_nsfw_model.git

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import os
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms, models
from PIL import Image
from torch.autograd import Variable

In [ ]:
#image directory
data_dir = '/kaggle/input/probe-set/probe/'

'''test_transforms = transforms.Compose([transforms.Resize(224),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                          std=[0.229, 0.224, 0.225])
                                     ])'''

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
'''model = models.resnet50()
model.fc = nn.Sequential(nn.Linear(2048, 512),
                                 nn.ReLU(),
                                 nn.Dropout(0),
                                 nn.Linear(512, 10),
                                 nn.LogSoftmax(dim=1))
if torch.cuda.is_available():
    model = model.to(device)
model.load_state_dict(torch.load('ResNet50_nsfw_model.pth'))
model.eval()'''

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
#prediction function
def predict_image(image):
    image_tensor = transformations(image).float()
    image_tensor = image_tensor.unsqueeze_(0)

    if torch.cuda.is_available():
        image_tensor = image_tensor.to(device)

    input = Variable(image_tensor)
    output = model(input)
    index = output.cpu().data.numpy().argmax()
    return index

In [ ]:
# #model classes
# classes=['drawings', 'hentai', 'neutral', 'porn', 'sexy']

# #load images
# entries = os.listdir(data_dir)

# fig=plt.figure(figsize=(10,10))
# i=0

# for entry in entries:
#     i+=1
#     image = Image.open(data_dir+entry)
    
#     #prediction
#     index = predict_image(image)
    
#     sub = fig.add_subplot(1, len(entries), i)
#     sub.set_title(classes[index])
#     plt.axis('off')
#     plt.imshow(image)
# plt.show()

In [ ]:
model = models.resnet50(pretrained=True)
model.eval()

In [ ]:

model = models.resnet50(pretrained=True)
in_features = model.fc.in_features


fc = nn.Linear(in_features = in_features, out_features = 3)
model.fc = fc
'''model.fc = nn.Sequential(nn.Linear(2048, 512),
                                 nn.ReLU(),
                                 nn.Dropout(0.05),
                                 nn.Linear(512, 3),
                                 nn.ReLU())'''
if torch.cuda.is_available():
    model = model.to(device)
#model.eval()

In [ ]:
model

In [ ]:
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True
# for param in model.layer2[3].parameters():
#     param.requires_grad = True
for param in model.layer4[2].parameters():
    param.requires_grad = True
for param in model.layer3[2].parameters():
    param.requires_grad = True

In [ ]:
weight = torch.tensor([1.5, 1.0, 1.0])
# weight = torch.tensor([1.0, 1.5, 1.0, 1.0, 1.5])
class_weights = torch.FloatTensor(weight).cuda()
criterion = nn.CrossEntropyLoss(weight=class_weights)
#criterion = nn.NLLLoss(weight=weight)
#criterion = nn.CrossEntropyLoss()
# finetune_optim = optim.Adam(model.parameters(),lr=10e-5)
# finetune_optim = optim.SGD(model.parameters(),lr=10e-4)
# finetune_optim = optim.ASGD(model.parameters(),lr=10e-4)
# finetune_optim = optim.Adagrad(model.parameters(),lr=10e-4)
finetune_optim = optim.Adadelta(model.parameters(),lr=10e-4)
# finetune_optim = optim.RMSprop(model.parameters(),lr=10e-4)
# finetune_optim = optim.Adamax(model.parameters(),lr=10e-4)

In [ ]:
train_loss = []
test_loss = []
train_acc = []
test_acc = []

In [ ]:
model = model_training(model, criterion, finetune_optim,
                       number_epochs=11)

In [ ]:
train_loss

In [ ]:
test_loss

In [ ]:
plt.figure(figsize=(10, 5))

plt.title("Loss", fontsize=25, fontweight="bold")
plt.plot(range(11),train_loss,label = 'train')
plt.plot(range(11),test_loss, label = 'test')
plt.xlabel("Epoch", fontsize=14, fontweight="bold")
plt.ylabel("Loss", fontsize=14, fontweight="bold")
plt.legend(["Train loss","Test loss"])
plt.show()

In [ ]:
#prediction function
def predict_image(image):
    image_tensor = transformations(image).float()
    image_tensor = image_tensor.unsqueeze_(0)

    if torch.cuda.is_available():
        image_tensor = image_tensor.to(device)

    input = Variable(image_tensor)
    output = model(input)
    index = output.cpu().data.numpy().argmax()
    return index

In [ ]:
# classes=['nude', 'people', 'safe']
# import torchvision.transforms as T
# #load images
# entries = test_loader

# for content,batch in entries:
    
#     fig=plt.figure(figsize=(20,20))
#     i=0
    
#     for entry in content:

#         #prediction
             
#         image = T.ToPILImage()(entry)
#         index_true = batch[i]
#         index = predict_image(image)
#         i+=1
#         sub = fig.add_subplot(1, len(batch), i)
#         sub.set_title(classes[index]+'/'+classes[index_true])
#         plt.axis('off')
#         plt.imshow(image)
#     plt.show()

In [ ]:
train_acc[0].data

In [ ]:
boxes = [x.cpu().numpy() for x in train_acc]
test_boxes = [x.cpu().numpy() for x in test_acc]
boxes

In [ ]:
test_boxes

In [ ]:
plt.figure(figsize=(10, 5))

plt.title("Accuracy", fontsize=25, fontweight="bold")
plt.plot(range(11),boxes,label = 'train')
plt.plot(range(11),test_boxes, label = 'test')
plt.xlabel("Epoch", fontsize=14, fontweight="bold")
plt.ylabel("Loss", fontsize=14, fontweight="bold")
plt.legend(["Train accuracy","Test accuracy"])
plt.show()

In [ ]:
torch.save(model.state_dict(), 'ResNet50_nude_model_adagrad.pth')

In [ ]:
# model = models.resnet50(pretrained=True)
# in_features = model.fc.in_features


# fc = nn.Linear(in_features = in_features, out_features = 3)
# model.fc = fc
# '''model.fc = nn.Sequential(nn.Linear(2048, 512),
#                                  nn.ReLU(),
#                                  nn.Dropout(0.05),
#                                  nn.Linear(512, 3),
#                                  nn.ReLU())'''
# if torch.cuda.is_available():
#     model = model.to(device)
# model.load_state_dict(torch.load('ResNet50_nsfw_model_0.pth'))
# model.eval()

In [ ]:
# inputs = test_transforms(img_collect[0][1])
# inputs
# if torch.cuda.is_available():
#     inputs = inputs.to(device)

In [ ]:
# model(inputs)

In [ ]:
# finetune_model.eval()

In [ ]:
# model.eval()

In [ ]:
# #prediction function
# def predict_image(image):
#     image_tensor = transformations(image).float()
#     image_tensor = image_tensor.unsqueeze_(0)

#     if torch.cuda.is_available():
#         image_tensor = image_tensor.to(device)

#     input = Variable(image_tensor)
#     output = model(input)
#     index = output.cpu().data.numpy().argmax()
#     return index

In [ ]:
# class_names

In [ ]:
# #model classes
# classes=['nude', 'safe_people', 'safe']

# data_dir = '/kaggle/input/probe-set/probe/'
# # data_dir = '/kaggle/input/final-nude-dataset/final_dataset_nood/test/nude/'


# #load images
# entries = os.listdir(data_dir)

# fig=plt.figure(figsize=(10,10))
# i=0

# for entry in entries:
#     i+=1
#     image = Image.open(data_dir+entry)
    
#     #prediction
#     print(image)
#     index = predict_image(image)
#     print(index)
    
#     sub = fig.add_subplot(1, len(entries), i)
#     sub.set_title(classes[index])
#     plt.axis('off')
#     plt.imshow(image)
# plt.show()

In [ ]:
# predict_image(img_collect[0][2])

In [ ]:
# model = models.resnet50(pretrained = True)
# model.fc = nn.Sequential(nn.Linear(2048, 512),
#                                  nn.ReLU(),
#                                  nn.Dropout(0.2),
#                                  nn.Linear(512, 5),
#                                  nn.LogSoftmax(dim=1))
# if torch.cuda.is_available():
#     device = torch.device('cuda')
#     model = model.to(device)
# # model.load_state_dict(torch.load('ResNet50_nsfw_model_0.pth'))
# model.eval()

In [ ]:
# #model classes
# classes=['drawings', 'hentai', 'neutral', 'porn', 'sexy']

# #load images
# entries = os.listdir(data_dir)

# fig=plt.figure(figsize=(10,10))
# i=0

# for entry in entries:
#     i+=1
#     image = Image.open(data_dir+entry)
    
#     #prediction
#     index = predict_image(image)
    
#     sub = fig.add_subplot(1, len(entries), i)
#     sub.set_title(classes[index])
#     plt.axis('off')
#     plt.imshow(image)
# plt.show()

**TreshNet**

In [ ]:
# !cp -r /kaggle/input/trash_dataset/trash_dataset /kaggle/working/